In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
from pathlib import Path
from compile_sweep_results import compile_sweep_results
from tqdm import tqdm

In [ ]:
root = Path("/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/symmetry_breaking/sweep_results/")
sweep_name = "sweep03_extinction"
output_dir = root / sweep_name

In [ ]:
rows = []

for path in tqdm(output_dir.glob("*.json")):
    # First try pandas' JSON reader for Series format
    s = pd.read_json(path, typ="series")
    s = s.copy()
    s["__source_file__"] = path.name
    s["sim_id"] = s["__source_file__"].replace("result_", "").replace(".json", "")
    rows.append(s)

### Compile results into pandas dataframes

In [ ]:
array_cols = ["Activator", "Repressor", "rho"]
skip_cols = ["x", "times"] + array_cols
param_cols = [col for col in rows[0].keys() if col not in ["sim_id"] + skip_cols]
param_cols = ["sim_id"] + param_cols

x_grid = (np.asarray(rows[0]["x"]) - 1500)[:, None]
t_grid = np.asarray(rows[0]["times"])[:, None]
x_cols = [f"x{i:04}" for i in range(len(x_grid))]

param_dfs = []
activator_dfs = []
repressor_dfs = []
rho_dfs = []
# iterate
for row in tqdm(rows):

    # params
    p_temp = pd.DataFrame(row[param_cols])
    param_dfs.append(p_temp.T)
    
    sim_id_vec = np.tile(row["sim_id"], len(t_grid))[:, None]
    # N df
    a_array = np.asarray(row["Activator"])
    a_temp = pd.DataFrame(np.c_[sim_id_vec, t_grid, a_array], columns=["sim_id", "t"] + x_cols)
    activator_dfs.append(a_temp)
    # L df
    r_array = np.asarray(row["Repressor"])
    r_temp = pd.DataFrame(np.c_[sim_id_vec, t_grid, r_array], columns=["sim_id", "t"] + x_cols)
    repressor_dfs.append(r_temp)
    # rho df
    rho_array = np.asarray(row["rho"])
    rho_temp = pd.DataFrame(np.c_[sim_id_vec, t_grid, rho_array], columns=["sim_id", "t"] + x_cols)
    rho_dfs.append(rho_temp)


Nodal_df = pd.concat(activator_dfs, axis=0, ignore_index=True)
Lefty_df = pd.concat(repressor_dfs, axis=0, ignore_index=True)
rho_df = pd.concat(rho_dfs, axis=0, ignore_index=True)
param_df = pd.concat(param_dfs, axis=0, ignore_index=True)

Nodal_df.head()

### Calculate profile stats/trajectories

In [ ]:
sigma_N = 10
mu_L = 0.0002275846
mu_N = mu_L * 1.11e-4/0.61e-4
# target domain sigma: 250
d_sigma = 250
ref_grid = x_grid.ravel()
# generate calculation masks
mid_mask = np.abs(ref_grid) <= d_sigma
left_mask = ref_grid < -d_sigma
right_mask = ref_grid > d_sigma

In [ ]:
# nodal_factor_vec = np.tile(np.asarray(np.divide(sigma_N, param_df["mu_L"]*,))[:, None], len(t_grid))
# lefty_factor_vec = np.tile(np.asarray(np.divide(param_df["sigma_L"], param_df["mu_L"]))[:, None], len(t_grid))

In [ ]:
nodal_array = Nodal_df.loc[:, x_cols].to_numpy().astype(float)
N_left = np.mean(nodal_array[:, left_mask==1], axis=1)
N_center = np.mean(nodal_array[:, mid_mask==1], axis=1)
N_right = np.mean(nodal_array[:, right_mask==1], axis=1)

lefty_array = Lefty_df.loc[:, x_cols].to_numpy().astype(float)
L_left = np.mean(lefty_array[:, left_mask==1], axis=1)
L_center = np.mean(lefty_array[:, mid_mask==1], axis=1)
L_right = np.mean(lefty_array[:, right_mask==1], axis=1)

In [ ]:
pattern_df = pd.DataFrame(Nodal_df.loc[:, "sim_id"]).merge(param_df, how="left", on="sim_id")
pattern_df["N_left"] = N_left
pattern_df["N_center"] = N_center 
pattern_df["N_right"] = N_right

pattern_df["L_left"] = L_left
pattern_df["L_center"] = L_center 
pattern_df["L_right"] = L_right

pattern_df.head()

In [ ]:
fig = px.scatter(pattern_df, x="N_center", y="N_left")
fig.show()

In [ ]:
pd.DataFrame(param_dfs)